In [3]:
import math

import numpy as np
import pandas as pd

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

funclass_dict = {"1":"Silent", "2":"Nonsense", "3":"Misense", "4":"None"}
af_dict = {"1":"AF < 0.20", "2": "0.80 > AF >= 0.20", "3":"AF >= 0.80"}

# Method to display human readable rule, where absolute RNA position is translated to relative Protein position. 
def protein_relative_position(pos, offset):
  return math.floor((pos - offset) / 3) + 1

# Method to display human readable rule, where absolute RNA position is translated to relative Protein position.
def to_str_each_translated(items, offset):
  items_str = ""
  for item in items:
    item_str = "(" + str(protein_relative_position(int(item[0:-2:1]), offset)) + ", " + funclass_dict.get(item[-2:-1:1]) + ", " + af_dict.get(item[-1]) + ") & "
    items_str += item_str 
  return items_str[:-3]

# Method to display human readable rule, where absolute RNA position is translated to relative Protein position.
def to_str_translated(ser, offset):    
  # Cast to string
  ser = ser.astype(str)  

  # Get rid of unnecessary characters prior to list of (position + funclass + af) numbers
  ser = ser.str.slice(12,-3,1)  

  # Get rid of single quotes
  ser = ser.replace('\'', '', regex=True)

  ser = ser.str.split(pat=",")

  return ser.apply(to_str_each_translated, args=(offset,))

# Method to display human readable rule, where absolute RNA position is translated to relative Protein position.
def add_readable_rules_translated(df_in, offset):
  # Empty data frame
  if df_in is None or df_in.shape[0] == 0:
    return df_in

  df = df_in.copy()

  antecedents_str = to_str_translated(df['antecedents'], offset)
  consequents_str = to_str_translated(df['consequents'], offset)

  readable_rule = "(" + antecedents_str + ") => (" + consequents_str + ")"
  df.insert(2,'translated_readable_rule', readable_rule)

  return df

# Method to display human readable rule
def to_str_each(items):
  items_str = ""
  for item in items:
    item_str = "(" + item[0:-2:1] + ", " + funclass_dict.get(item[-2:-1:1]) + ", " + af_dict.get(item[-1]) + ") & "
    items_str += item_str 
  return items_str[:-3]

# Method to display human readable rule
def to_str(ser):    
  # Cast to string
  ser = ser.astype(str)  

  # Get rid of unnecessary characters prior to list of (position + funclass + af) numbers
  ser = ser.str.slice(12,-3,1)  

  # Get rid of single quotes
  ser = ser.replace('\'', '', regex=True)

  ser = ser.str.split(pat=",")

  return ser.apply(to_str_each)

# Method to display human readable rule
def add_readable_rules(df_in):
  # Empty data frame
  if df_in is None or df_in.shape[0] == 0:
    return df_in

  df = df_in.copy()

  antecedents_str = to_str(df['antecedents'])
  consequents_str = to_str(df['consequents'])

  readable_rule = "(" + antecedents_str + ") => (" + consequents_str + ")"
  df.insert(2,'readable_rule', readable_rule)

  return df

# Filter data based on probability of position in the dataset. Filter specified as a range.
def filter_on_position_probability(df_in, start_prob=None, end_prob=None):
    if df_in is None or df_in.shape[0] == 0:
      return df_in

    if start_prob is None and end_prob is None:
      return df_in

    # Only consider rows where probability of Position being present in the Samples is >= start_prob and <= end_prob
    num_samples = df_in['Sample'].nunique()
    print("num_samples: {}".format(num_samples))

    num_positions = df_in['POS'].nunique()
    print("num_positions: {}".format(num_positions))
   
    df = df_in[['Sample', 'POS']].groupby(['POS']).agg(position_prob=('Sample', 'count'))
    # Normalize position_prob by dividing it by num_samples. This makes it a proability between 0.0 and 1.0
    df['position_prob'] = df['position_prob'] / num_samples   

    df_sorted = df.sort_values(by = 'position_prob')
    print(df_sorted.head())
    print(df_sorted.tail())
  
    if start_prob is not None:
      df = df[df.position_prob >= start_prob]

    if end_prob is not None:
      df = df[df.position_prob <= end_prob]
    
    ret_val = pd.merge(df_in, df, on='POS')

    df_sorted = ret_val.sort_values(by = 'position_prob')[['POS', 'position_prob']]
    print(df_sorted.head())
    print(df_sorted.tail())

    return ret_val

# Return the probaility of a position. For testing/verification purposes.
def get_position_probability(df_in, position=None):
    if df_in is None or df_in.shape[0] == 0:
      return df_in

    if position is None:
      return df_in

    num_samples = df_in['Sample'].nunique()
    print("num_samples: {}".format(num_samples))

    num_positions = df_in['POS'].nunique()
    print("num_positions: {}".format(num_positions))
   
    df = df_in[['Sample', 'POS']].groupby(['POS']).agg(position_prob=('Sample', 'count'))
    # Normalize position_prob by dividing it by num_samples. This makes it a proability between 0.0 and 1.0
    df['position_prob'] = df['position_prob'] / num_samples

    try:      
      return df.loc[position,'position_prob']
    except KeyError:
      return -1

# Filter data based on values of position, AF, and funclass. Position and AF filter specified as a range. funclass filter is a list of acceptable values.
def filter_on_column_values(df_in, start_pos=None, end_pos=None, start_af=None, end_af=None, funclass=[]):  
  if df_in is None or df_in.shape[0] == 0:
    return df_in

  df = df_in.copy()

  # Replace "." with "NONE" in FUNCLASS column. They both represent "Non-coding" variant
  df["FUNCLASS"] = df["FUNCLASS"].replace('.', 'NONE')

  # Filter in_file rows
  if start_pos is not None:
    df = df[df.POS >= start_pos]    
  if end_pos is not None:
    df = df[df.POS < end_pos]    
  if start_af is not None:
    df = df[df.AF >= start_af]    
  if end_af is not None:
    df = df[df.AF < end_af]    
  if len(funclass) > 0:    
    df = df[df.FUNCLASS.isin(funclass)]

  return df

# Create a single integer representing a variant at a specific position with a specific allele frequency
# Pivot the data so we have all sample variants on a single line
#
# Extract rows from in_file where
# 
#    POS >= start_pos & POS <= end_pos
#        If start_pos = None: POS <= end_pos
#        If end_pos = None: POS >= start_pos 
#
#    AF >= start_af & AF <= end_af
#        If start_af = None: AF <= end_af
#        If end_af = None: AF >= start_af
#
#    FUNCLASS in funclass (funclass is a list)      
#
def preprocess_input_file(df_in):
  if df_in is None or df_in.shape[0] == 0:
    return df_in
  
  df = df_in.copy()

  # Bucket values in FUNCLASS and AF columns. We do not bucket the values in POS column.

  # Replace variants in FUNCLASS column with a distinct numeric value
  df.loc[df.FUNCLASS == "NONE", "FUNCLASS"] = 4
  df.loc[df.FUNCLASS == "MISSENSE", "FUNCLASS"] = 3
  df.loc[df.FUNCLASS == "NONSENSE", "FUNCLASS"] = 2 
  df.loc[df.FUNCLASS == "SILENT", "FUNCLASS"] = 1 

  # Replace allele frequency in AF column with a distict numeric value
  df.loc[df.AF >= 0.80, "AF"] = 3
  df.loc[(df.AF >= 0.20) & (df.AF < 0.80), "AF"] = 2
  df.loc[df.AF < 0.20, "AF"] = 1

  # Convert AF values to integer
  df = df.astype({"AF": int}) 

  # Create a new column called 'Label', which is a string concatentation of POS, FUNCLASS, and AF values. 
  # The idea is to represent each variant + allele frequency + position as a single integer, to be used in MBA 
  df["Label"] = df["POS"].astype(str) + df["FUNCLASS"].astype(str) + df["AF"].astype(str)

  # We do not need POS, FUNCLASS, and AF columns anymore
  df = df[["Sample", "Label"]]
  
  # Add a new column called 'Value', prepopulated with 1
  df["Value"] = 1

  df = pd.pivot_table(df, index="Sample", columns="Label", values="Value")

  # Set all data frame nan (not a number) values to 0
  df = df.fillna(0)

  # Convert all data framevalues to integer
  df = df.astype(int) 

  return df

In [4]:
def get_association_rules(in_file, 
                          min_support=0.20, 
                          min_confidence=0.80, 
                          min_lift=1.0, 
                          min_conviction=1.0, 
                          max_len=None, 
                          start_pos=None, 
                          end_pos=None, 
                          start_af=None, 
                          end_af=None, 
                          funclass=[], 
                          start_prob=None, 
                          end_prob=None):
  # Read the input file and pick the needed columns
  df_in = pd.read_csv(in_file, sep='\t')[['Sample', 'POS', 'AF', 'FUNCLASS']]

  # Filter on position probability
  df_pp = filter_on_position_probability(df_in, start_prob, end_prob)

  # Filter on column values
  df_cv = filter_on_column_values(df_pp, start_pos, end_pos, start_af, end_af, funclass)

  # Preprocess the data frame
  df = preprocess_input_file(df_cv)

  # Get frequent item sets, with support larger than min_support, using Apriori algorithm
  frequent_itemsets = apriori(df, min_support=min_support, use_colnames=True, max_len=max_len)

  # Get association rules, with lift larger than min_lift  
  rules = association_rules(frequent_itemsets, metric="lift", min_threshold=min_lift)

  # Filter association rules, keeping rules with confidence larger than min_confidence
  rules = rules[ (rules['confidence'] >= min_confidence) & (rules['conviction'] >= min_conviction) ]

  return rules

In [63]:
# Original analysis
pd.set_option('max_columns', 10, 'display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', None)

rules = get_association_rules(in_file="https://github.com/galaxyproject/SARS-CoV-2/raw/master/data/var/bos_by_sample.tsv.gz", 
                            min_support=0.1, 
                            min_confidence=0.9, 
                            #min_lift=2, 
                            #min_conviction=2.0,
                            #start_af=0.2,
                            #end_af=0.8,
                            #funclass=['MISSENSE','SILENT'],
                            max_len=8,
                            #start_prob=.5
                            )
num_rules = rules.shape[0]
print('Boston dataset association rules: ')
br = add_readable_rules(rules)
print(rules.head(num_rules))
print('\n\n')


Boston dataset association rules: 
                                antecedents                          consequents  antecedent support  consequent support   support  confidence      lift  leverage  conviction
0                                   (10543)                            (1440833)            0.134585            0.951487  0.134585    1.000000  1.050987  0.006529         inf
2                                 (1887713)                              (10543)            0.145540            0.134585  0.134585    0.924731  6.870968  0.114998   11.497653
3                                   (10543)                            (1887713)            0.134585            0.145540  0.134585    1.000000  6.870968  0.114998         inf
5                                   (10543)                            (2340333)            0.134585            0.949922  0.134585    1.000000  1.052718  0.006740         inf
7                                   (10543)                              (24143)          

In [64]:
d = {}
i = 0
row_list = []
for row in rules[['antecedents','consequents']].iterrows():
    for a_or_c in ['antecedents','consequents']:
        for item in row[1][a_or_c]:
            row_list.append(row[0])
            row_list.append(a_or_c[0])
            row_list.append(int(item[:len(item)-2]))
            row_list.append(int(item[-2]))
            row_list.append(int(item[-1]))
            d[i]=row_list
            i+=1
            row_list = []
rule_narrow = pd.DataFrame.from_dict(d,orient='index',columns=['rule','type','pos','func','af'])

In [67]:
from bokeh.plotting import figure, show 
from bokeh.io import output_notebook
from math import pi

output_notebook()

p = figure(plot_width=1200, 
           plot_height=800,
           #x_range=[str(x) for x in sorted(rule_narrow['pos'].unique())]
           )

# add a circle renderer with a size, color, and alpha
p.circle(x='pos',y='rule',source=rule_narrow[rule_narrow['type']=='a'], color="red", size=10,alpha=0.5)
p.circle(x='pos',y='rule',source=rule_narrow[rule_narrow['type']=='c'], color="green", alpha=0.5)
p.xaxis.ticker = sorted(rule_narrow['pos'].unique())
#p.yaxis.ticker = sorted(rule_narrow['rule'].unique())
p.yaxis.axis_label = "Rule #"
p.xaxis.axis_label = "Position"
p.xaxis.major_label_orientation = pi/2



# show the results
show(p)

In [ ]:
# Test get_position_probability()

pd.set_option('max_columns', 10, 'display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', None)

# Read the input file and pick the needed columns
in_file = "https://github.com/galaxyproject/SARS-CoV-2/raw/master/data/var/bos_by_sample.tsv.gz"
df_in = pd.read_csv(in_file, sep='\t')[['Sample', 'POS', 'AF', 'FUNCLASS']]

position=26542
position_prob = get_position_probability(df_in, position=position)
print("position: {}, position_prob: {:.4f}".format(position, position_prob))

position=26542
position_prob = get_position_probability(df_in, position=position)
print("position: {}, position_prob: {:.4f}".format(position, position_prob))

position=23403
position_prob = get_position_probability(df_in, position=position)
print("position: {}, position_prob: {:.4f}".format(position, position_prob))


num_samples: 639
num_positions: 1013
position: 26542, position_prob: 0.2598
num_samples: 639
num_positions: 1013
position: 26542, position_prob: 0.2598
num_samples: 639
num_positions: 1013
position: 23403, position_prob: 0.9499
